In [1]:
import re
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import col, regexp_replace, trim,when ,monotonically_increasing_id,lit,year, month, dayofmonth, weekofyear, dayofweek, date_format,floor,dense_rank,\
substring,concat,split, row_number
from pyspark.sql.window import Window
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
from datetime import date, datetime, timedelta
import subprocess
from py4j.java_gateway import java_import
import os
from pyspark.sql.types import DateType

In [2]:
spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("sales_transactions")\
    .config("spark.eventLog.logBlockUpdates.enabled", True)\
    .enableHiveSupport()\
    .getOrCreate()

sc = spark.sparkContext

In [3]:
now = datetime.now()
date_str = now.strftime("%Y%m%d")
hour_str = now.strftime("%H")
print(date_str, hour_str)

20240707 22


In [39]:
input_trans = spark.read.csv(f"hdfs:///data/retail_silver/{date_str}/{hour_str}/sales_transactions_SS_cleaned_{date_str}_{hour_str}.csv", header='true')

In [7]:
#function_to_rename_in_hdfs
def rename_in_hdfs(golden_layer_path,file_extension,name):
    # Run the Hadoop fs -ls command to list files
    list_files_process = subprocess.run(["hadoop", "fs", "-ls", golden_layer_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Check for errors
    if list_files_process.returncode != 0:
        print(f"Error listing files in {golden_layer_path}: {list_files_process.stderr.decode()}")
        exit(1)

    # Decode stdout to string format and split lines
    stdout_str = list_files_process.stdout.decode()
    file_list = stdout_str.splitlines()

    # Find the file to rename based on criteria
    file_to_rename = None
    for line in file_list:
        if line.endswith(file_extension):
            file_to_rename = line.split()[-1].strip()
            break

    # Check if a file matching the criteria was found
    if file_to_rename:
        new_filename = f"{golden_layer_path}/{name}_{date_str}_{hour_str}{file_extension}"

        # Move (rename) the file
        subprocess.run(["hadoop", "fs", "-mv", file_to_rename, new_filename])

        print(f"File moved and renamed to: {new_filename}")
    else:
        print("File matching the criteria not found.")

In [8]:
#write customer dim in HDFS
cust_dim = input_trans.dropDuplicates(['customer_id'])

# Add a sequential surrogate key column
window_spec = Window.orderBy("customer_id")
cust_dim = cust_dim.withColumn('customer_sur_key', row_number().over(window_spec))

#to write cust_dim in one file 
cust_dim = cust_dim.repartition(1)
golden_layer_path="hdfs:///data/golden_layer/cust_dim"
file_extension = ".csv"
name='cust_dim'

#make customer dim 
cust_dim = cust_dim.select('customer_sur_key','customer_id', 'customer_fname', 'cusomter_lname', 'customer_email') 
cust_dim.write.mode('overwrite') \
        .option("header", "true") \
        .format('csv') \
        .save(golden_layer_path)
cust_dim.show(5)

# Define your directory path and file criteria
#directory_path = customer_dim_path
#golden_layer_path="hdfs:///data/golden_layer"
file_extension = ".csv"
name='cust_dim'
rename_in_hdfs(golden_layer_path,file_extension,name)

+----------------+-----------+--------------+--------------+--------------------+
|customer_sur_key|customer_id|customer_fname|cusomter_lname|      customer_email|
+----------------+-----------+--------------+--------------+--------------------+
|               1|      85462|           Mia|         Davis|mia.davis@outlook...|
|               2|      85463|           Ava|        Miller|ava.miller@gmail.com|
|               3|      85464|     Alexander|         Moore|alexander.moore@o...|
|               4|      85465|          Emma|         Smith|emma.smith@gmail.com|
|               5|      85466|       William|         Moore|william.moore@hot...|
+----------------+-----------+--------------+--------------+--------------------+
only showing top 5 rows

File moved and renamed to: hdfs:///data/golden_layer/cust_dim/cust_dim_20240707_22.csv


In [9]:
# Ensure product_dim is distinct by product_id and add a sequential surrogate key
product_dim = input_trans.dropDuplicates(['product_id'])
window_spec = Window.orderBy("product_id")
product_dim = product_dim.withColumn('product_sur_key', row_number().over(window_spec))

# Repartition to one file for efficient writing
product_dim = product_dim.repartition(1)

# Define the golden layer path and file details
golden_layer_path = "hdfs:///data/golden_layer/product_dim"
file_extension = ".csv"
name = 'product_dim'

# Select relevant columns for product_dim
product_dim = product_dim.select('product_sur_key', 'product_id', 'product_name', 'product_category')

# Write product_dim to HDFS in CSV format
product_dim.write.mode('overwrite') \
            .option("header", "true") \
            .format('csv') \
            .save(golden_layer_path)

# Show the first 5 rows of product_dim (optional)
product_dim.show(5)

# Rename the file in HDFS if necessary
rename_in_hdfs(golden_layer_path, file_extension, name)

+---------------+----------+------------+----------------+
|product_sur_key|product_id|product_name|product_category|
+---------------+----------+------------+----------------+
|              1|         1|      Laptop|     Electronics|
|              2|        10|     Sandals|        Footwear|
|              3|        11|          TV|     Electronics|
|              4|        12|     Monitor|     Electronics|
|              5|        13|     Printer|     Electronics|
+---------------+----------+------------+----------------+
only showing top 5 rows

File moved and renamed to: hdfs:///data/golden_layer/product_dim/product_dim_20240707_22.csv


In [10]:
# Define the file path for the initial CSV data and the golden layer path on HDFS
file_path = f"hdfs:///data/retail_bronze/{date_str}/{hour_str}/branches_SS_raw_{date_str}_{hour_str}.csv"
golden_layer_path = "hdfs:///data/golden_layer/branches_dim"
file_extension = ".csv"
name = "branches_dim"

# Load the CSV data into a PySpark DataFrame
branches_dim = spark.read.option("header", "true").csv(file_path)

# Convert establish_date to date type if needed
branches_dim = branches_dim.withColumn("establish_date", col("establish_date").cast("date"))

# Add a sequential surrogate key column
window_spec = Window.orderBy("branch_id")
branches_dim = branches_dim.withColumn('branch_sur_key', row_number().over(window_spec))

# Drop duplicates based on branch_id if necessary
branches_dim = branches_dim.dropDuplicates(['branch_id'])

# Show the DataFrame with the surrogate key if needed
# branches_dim.show()

# Save the DataFrame to the golden layer folder on HDFS in CSV format
branches_dim.write.option("header", "true").mode("overwrite").csv(golden_layer_path)
rename_in_hdfs(golden_layer_path, file_extension, name)

File moved and renamed to: hdfs:///data/golden_layer/branches_dim/branches_dim_20240707_22.csv


In [11]:
# Define the file path for the initial CSV data and the golden layer path on HDFS
file_path = f"hdfs:///data/retail_bronze/{date_str}/{hour_str}/sales_agents_SS_raw_{date_str}_{hour_str}.csv"
golden_layer_path = "hdfs:///data/golden_layer/sales_agent_dim"
file_extension = ".csv"
name = "sales_agent"

# Load the CSV data into a PySpark DataFrame
agent_dim = spark.read.option("header", "true").csv(file_path)

# Convert hire_date to date type if needed
agent_dim = agent_dim.withColumn("hire_date", col("hire_date").cast("date"))

# Add a sequential surrogate key column
window_spec = Window.orderBy("sales_person_id")
agent_dim = agent_dim.withColumn('sales_agent_sur_key', row_number().over(window_spec))

# Show the DataFrame with the surrogate key
agent_dim.show(5)

# Save the DataFrame to the golden layer folder on HDFS in CSV format
agent_dim.write.option("header", "true").mode("overwrite").csv(golden_layer_path)

# Rename the file in HDFS if necessary
rename_in_hdfs(golden_layer_path, file_extension, name)

+---------------+---------------+----------+-------------------+
|sales_person_id|           name| hire_date|sales_agent_sur_key|
+---------------+---------------+----------+-------------------+
|              1|       John Doe|2020-06-10|                  1|
|             10|   Sophia Moore|2019-11-10|                  2|
|             11|      john wick|2018-07-10|                  3|
|              2|     Jane Smith|2021-06-08|                  4|
|              3|Michael Johnson|2019-07-22|                  5|
+---------------+---------------+----------+-------------------+
only showing top 5 rows

File moved and renamed to: hdfs:///data/golden_layer/sales_agent_dim/sales_agent_20240707_22.csv


In [15]:
#create date dimension
# Generate date range

# Generate date range
start_date = date(2022, 1, 1)
end_date = date(2024, 12, 31)

date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
date_df = spark.createDataFrame([(d,) for d in date_range], ["date"]).withColumn("date", col("date").cast("date"))

# Add date attributes
date_dim = date_df.withColumn("year", year(col("date"))) \
    .withColumn("month", month(col("date"))) \
    .withColumn("day", dayofmonth(col("date"))) \
    .withColumn("week", weekofyear(col("date"))) \
    .withColumn("weekday", dayofweek(col("date"))) \
    .withColumn("quarter", floor((month(col("date")) - 1) / 3) + 1) \
    .withColumn("day_name", date_format(col("date"), "EEEE")) \
    .withColumn("month_name", date_format(col("date"), "MMMM")) \
    .withColumn("is_weekend", when(col("weekday").isin([1, 7]), lit(1)).otherwise(lit(0)))

# Add surrogate key column
date_dim = date_dim.withColumn("date_sur_key", concat(col('day'),col('month'),col('year')))


# Define the output directory for the date dimension
date_dim_path = "hdfs:///data/golden_layer/date_dim"

try:
    # Write the date dimension to a single CSV file
    date_dim.repartition(1) \
        .write.mode('overwrite') \
        .option("header", "true") \
        .format('csv') \
        .save(date_dim_path)
    print(f"Date dimension table saved to {date_dim_path}")
except Exception as e:
    print(f"An error occurred: {e}")
    
# to rename csv file in date dim
file_extension = ".csv"
name="date_dim"

rename_in_hdfs(date_dim_path, file_extension, name)

Date dimension table saved to hdfs:///data/golden_layer/date_dim
File moved and renamed to: hdfs:///data/golden_layer/date_dim/date_dim_20240707_22.csv


In [36]:
# fact One (offline)

# Filter and transform input_trans for offline transactions
branch_transaction_fact = input_trans.filter(col('is_online') == "no")
columns_to_drop = ['shipping_address', 'customer_fname', 'customer_lname',
                   'offer_1', 'offer_2', 'offer_3', 'offer_4', 'offer_5',
                   'product_name', 'product_category', 'customer_email']
branch_transaction_fact = branch_transaction_fact.drop(*columns_to_drop)

# Convert 'transaction_date' to DateType
branch_transaction_fact = branch_transaction_fact.withColumn("transaction_date", col("transaction_date").cast(DateType()))

# Calculate total_price
final_price = (col('units') * col('unit_price') * (1 - col('discount_perc') / 100))
branch_transaction_fact = branch_transaction_fact.withColumn("total_price", final_price)

# Join with dimension tables
branch_transaction_fact = branch_transaction_fact.join(cust_dim, on='customer_id', how='left') \
                           .join(product_dim, on='product_id', how='left') \
                           .join(date_dim, date_dim.date == branch_transaction_fact.transaction_date, 'left') \
                           .join(agent_dim, agent_dim.sales_person_id == branch_transaction_fact.sales_agent_id, 'left') \
                           .join(branches_dim, branches_dim.branch_id == branch_transaction_fact.branch_id, 'left')

branch_transaction_fact = branch_transaction_fact.select(
    'transaction_id',
    'branch_sur_key',
    'product_sur_key',
    'customer_sur_key',
    'sales_agent_sur_key',
    'date_sur_key',
    'units',
    'unit_price',
    'discount_perc',
    'total_price',
    'payment_method'
)

# Define output path for the fact table
fact_off_dim_path = "hdfs:///data/golden_layer/branch_transaction_fact"
file_extension = ".csv"

try:
    # Repartition and write the fact table to a single CSV file
    offline_fact.repartition(1) \
                .write.mode('overwrite') \
                .option("header", "true") \
                .format('csv') \
                .save(fact_off_dim_path)

    # Rename CSV file in HDFS
    name = "branch_transaction_fact"
    rename_in_hdfs(fact_off_dim_path, file_extension, name)
    print(f"Offline fact table saved to {fact_off_dim_path}/{name}{file_extension}")

except Exception as e:
    print(f"An error occurred: {e}")

File moved and renamed to: hdfs:///data/golden_layer/branch_transaction_fact/branch_transaction_fact_20240707_22.csv
Offline fact table saved to hdfs:///data/golden_layer/branch_transaction_fact/branch_transaction_fact.csv


In [38]:
#online_fact

# Filter and transform input_trans for online transactions
online_transaction_fact = input_trans.filter(col('is_online') == "yes")
columns_to_drop = ['customer_fname', 'cusomter_lname', 'sales_agent_id', 'branch_id', 'offer_1', 'offer_2',
                   'offer_3', 'offer_4', 'offer_5', 'product_name', 'product_category', 'customer_email']
online_transaction_fact = online_transaction_fact.drop(*columns_to_drop)#print(online_fact.columns)

# Convert 'transaction_date' to DateType
online_transaction_fact = online_transaction_fact.withColumn("transaction_date", col("transaction_date").cast(DateType()))

# Calculate total_price
final_price = (col('units') * col('unit_price') * (1 - col('discount_perc') / 100))
online_transaction_fact = online_transaction_fact.withColumn("total_price", final_price)

# Process 'shipping_address' column to split into separate columns
split_address_col = split(col("shipping_address"), '/')
online_transaction_fact = online_transaction_fact.withColumn('street', split_address_col.getItem(0)) \
                                                 .withColumn('city', split_address_col.getItem(1)) \
                                                 .withColumn('state', split_address_col.getItem(2)) \
                                                 .withColumn('postal_code', split_address_col.getItem(3))

# Join with dimension tables using left join
online_transaction_fact = online_transaction_fact.join(cust_dim, on='customer_id', how='left') \
                                                 .join(product_dim, on='product_id', how='left') \
                                                 .join(date_dim, date_dim.date == online_transaction_fact.transaction_date, 'left')

# Select relevant columns for the fact table
online_transaction_fact = online_transaction_fact.select(
    'transaction_id',
    'units',
    'payment_method',
    'discount_perc',
    'total_price',
    'customer_sur_key',
    'product_sur_key',
    'date_sur_key',
    'street',
    'city',
    'state',
    'postal_code'
)

# Define output path for the fact table
online_fact_path = "hdfs:///data/golden_layer/online_transaction_fact"
file_extension = ".csv"

try:
    # Repartition and write the fact table to a single CSV file
    online_fact.repartition(1) \
                .write.mode('overwrite') \
                .option("header", "true") \
                .format('csv') \
                .save(online_fact_path)

    # Rename CSV file in HDFS
    name = "online_transaction_fact"
    rename_in_hdfs(online_fact_path, file_extension, name)
    print(f"Online fact table saved to {online_fact_path}/{name}{file_extension}")

except Exception as e:
    print(f"An error occurred: {e}")

File moved and renamed to: hdfs:///data/golden_layer/online_transaction_fact/online_transaction_fact_20240707_22.csv
Online fact table saved to hdfs:///data/golden_layer/online_transaction_fact/online_transaction_fact.csv
